In [ ]:
import os
import re
import io
import sys
import glob
import enum
import json
import dask
import xlrd
import base64
import time
import shutil
import imageio
import requests
import datetime
import urllib
import psycopg2
import numpy as np
import pandas as pd
import skimage

import tifffile
import seaborn as sns
import matplotlib as mpl

import dask.diagnostics
import sqlalchemy as db
import sqlalchemy.orm
import sqlalchemy.ext.declarative
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('../..')
# %aimport opencell.imaging.managers
# %aimport opencell.imaging.processors

import opencell
from opencell import constants, file_utils
from opencell.cli import database_cli
from opencell.cli import fov_cli
from opencell.api import settings
from opencell.database import models
from opencell.database import operations
from opencell.database import utils as db_utils
from opencell.imaging import utils as im_utils
from opencell.imaging import images, managers, processors, viz

In [ ]:
url = db_utils.url_from_credentials('../../db-credentials-test.json')
# url = db_utils.url_from_credentials('../../db-credentials-dev.json')
# url = db_utils.url_from_credentials('../../db-credentials-cap.json')

engine = db.create_engine(url)
session_factory = db.orm.sessionmaker(bind=engine)
Session = db.orm.scoped_session(session_factory)
url

### Copy the raw TIFF for a given `fov_id` to the test directory

This is used to add FOVs to the local test versions of the `PlateMicroscopy` and `raw-pipeline-microscopy` directories. For this to work, `Session` must point to a production database (not a test database). 

After new FOVs are added to the test directories, the script `test-imaging-cli.sh` should be run to insert the FOVs into the test database and process them. 

Note that, for PMLs, the `fov-metadata.csv` files must be manually edited to remove all rows except for those corresponding to TIFFs copied here from ESS (which is invariably almost all of the rows). 

In [ ]:
fov = Session.query(models.MicroscopyFOV).filter(models.MicroscopyFOV.id == 34401).first()
p = processors.FOVProcessor.from_database(fov)

In [ ]:
p.src_filepath(), p.dst_filepath(kind='proj')

In [ ]:
src_dirpath, sirc_filename = os.path.split(p.src_filepath())

In [ ]:
# paths for PML datasets
ess_src_root = '/Volumes/ml_group/raw-pipeline-microscopy/'
local_src_root = '/Users/keith.cheveralls/opencell-test/raw-pipeline-microscopy/'

In [ ]:
# paths for legacy ML datasets 
ess_src_root = '/Volumes/ml_group/PlateMicroscopy/'
local_src_root = '/Users/keith.cheveralls/opencell-test/PlateMicroscopy/'

In [ ]:
os.makedirs(os.path.join(local_src_root, src_dirpath))

In [ ]:
# copy the raw TIFF stack
shutil.copy2(
    os.path.join(ess_src_root, p.src_filepath()),
    os.path.join(local_src_root, p.src_filepath())
)

In [ ]:
# copy the FOV metadata (for PMLs only, not legacy MLs)
pml_id = 'PML0320'
shutil.copy2(
    os.path.join(ess_src_root, pml_id, 'metadata.json'),
    os.path.join(local_src_root, pml_id, 'metadata.json')
)

shutil.copy2(
    os.path.join(ess_src_root, pml_id, 'fov-metadata.csv'),
    os.path.join(local_src_root, pml_id, 'fov-metadata.csv')
)

In [ ]:
# drop rows in the fov-metadata
pml_id = 'PML0340'
root = os.path.join(local_src_root, pml_id)
d = pd.read_csv(os.path.join(root, 'fov-metadata.csv'))
filenames = [path.split(os.sep)[-1] for path in glob.glob(os.path.join(root, 'raw_data', '*.tif'))]

In [ ]:
d_cropped = d.loc[d.src_filename.isin(filenames)]
d_cropped.to_csv(os.path.join(root, 'fov-metadata.csv'), index=False)

### Get an FOV from the test database

In [ ]:
config = opencell.api.settings.get_config('test')

In [ ]:
# FOV for a given target_name
line = operations.PolyclonalLineOperations.from_target_name(Session, 'ATL2').line
fov = line.fovs[0]
len(line.fovs)

In [ ]:
# FOV given an fov_id
fov = Session.query(models.MicroscopyFOV).filter(models.MicroscopyFOV.id == fov_id).first()

In [ ]:
# FOVProcessor given an roi_id
roi_id = 127647
fov = (
    Session.query(models.MicroscopyFOVROI)
    .filter(models.MicroscopyFOVROI.id == roi_id)
    .first()
    .fov
)

In [ ]:
p = processors.FOVProcessor.from_database(fov)
p.set_src_roots(
    plate_microscopy_dir=config.PLATE_MICROSCOPY_DIR, 
    raw_pipeline_microscopy_dir=config.RAW_PIPELINE_MICROSCOPY_DIR
)

In [ ]:
p.src_filepath(), p.dst_filepath(kind='proj')

In [ ]:
result = p.crop_annotated_roi(config.OPENCELL_MICROSCOPY_DIR)

### Debug annotated ROI cropping

In [ ]:
tiff = p.load_raw_tiff()

In [ ]:
plt.imshow(tiff.stacks['488'][25, :, :])

In [ ]:
aligned_stacks, result = tiff.align_cell_layer(-4, 4, 0.2, 1)

In [ ]:
result

In [ ]:
roi_props = p.crop_and_save_roi(
    roi_props, aligned_stacks, dst_root=config.OPENCELL_MICROSCOPY_DIR
)

In [ ]:
# hard-coded ROI props for an ROI (generated by crop_annotated_roi and passed to crop_and_save_roi)
roi_props = {
    'shape': (600, 600, 55),
    'position': (0, 0, 0),
    'xy_coords': (0, 0, 600, 600),
    'target_step_size': 0.4,
    'original_step_size': 0.2,
    'required_num_slices': 27,
}

### Internals of crop_and_save_roi for one channel

In [ ]:
def stack_to_uint8(stack, pmin, pmax):

    stack = stack.astype(float)
    minn, maxx = np.percentile(stack, (pmin, pmax))
    if minn == maxx:
        maxx = minn + 1

    stack -= minn
    stack[stack < 0] = 0
    stack /= (maxx - minn)
    stack[stack > 1] = 1

    stack = (255*stack).astype('uint8')
    return stack, int(minn), int(maxx)

In [ ]:
# this method mimics FOVProcessor.crop_and_save_roi

def crop_roi(aligned_stacks, roi_props, channel):

    num_rows, num_cols, num_z = roi_props['shape']
    row_ind, col_ind, z_ind = roi_props['position']

    stack = aligned_stacks[channel]
    cropped_stack = stack[
        z_ind:(z_ind + num_z),
        row_ind:(row_ind + num_rows),
        col_ind:(col_ind + num_cols)
    ].copy()

    # move the z dimension from the first to the last axis
    cropped_stack = np.moveaxis(cropped_stack, 0, -1)

    # resample the stack in z so it has the required step size and number of z-slices
    cropped_stack, did_resample_stack = p.maybe_resample_stack(
        cropped_stack,
        original_step_size=roi_props['original_step_size'],
        target_step_size=roi_props['target_step_size'],
        required_num_slices=roi_props['required_num_slices']
    )


    # downsample the pixel intensities from uint16 to uint8
    cropped_stack, min_intensity, max_intensity = stack_to_uint8(
        cropped_stack, pmin=0.01, pmax=(100 - .01)
    )

    if channel == '405':
        cropped_stack = skimage.filters.gaussian(
            cropped_stack, sigma=(.5, .5, .5), preserve_range=True
        )

    # save the stack itself as a one-dimensional tile of z-slices
    cropped_stack = np.moveaxis(cropped_stack, -1, 0).astype('uint8')
    tile = np.concatenate([zslice for zslice in cropped_stack], axis=0)
    return tile

In [ ]:
tile = crop_roi(aligned_stacks, roi_props, channel='488')

In [ ]:
imageio.imsave('test.jpg', tile, format='jpg', quality=100)
os.stat('test.jpg').st_size/1024/1024

In [ ]:
# generate the GFP ROI tile given a target name
# (requires that there's an annotated FOV for the target in the test image data)
def gfp_roi_tile_from_target_name(target_name):

    line = operations.PolyclonalLineOperations.from_target_name(Session, target_name).line
    fov = line.fovs[0]

    p = processors.FOVProcessor.from_database(fov)
    p.set_src_roots(
        plate_microscopy_dir=config.PLATE_MICROSCOPY_DIR, 
        raw_pipeline_microscopy_dir=config.RAW_PIPELINE_MICROSCOPY_DIR
    )

    tiff = p.load_raw_tiff()
    aligned_stacks, result = tiff.align_cell_layer(-5, 6, 0.2, 1)
    roi_props = {
        'shape': (600, 600, 55),
        'position': (0, 0, 0),
        'xy_coords': (0, 0, 600, 600),
        'target_step_size': 0.4,
        'original_step_size': 0.2,
        'required_num_slices': 27,
    }

    tile = crop_roi(aligned_stacks, roi_props, channel='488')
    return tile

In [ ]:
tile = roi_tile_from_target_name('MTOR')

In [ ]:
imageio.imsave('test.jpg', tile, format='jpg', quality=95)
os.stat('test.jpg').st_size/1024/1024

In [ ]:
# test the save_jpg method
p.save_jpg('test.jpg', tile, 1.5, min_quality=30, max_quality=95)

### Plot filesize vs JPG quality for a few different targets

In [ ]:
# POLR2F and LMNB1 are bright
# LTV1 and MTOR are diffuse/noisy, and TERT is just background
target_names = ['KRT18', 'TERT', 'LTV1', 'MTOR', 'LMNB1', 'POLR2F']

In [ ]:
tiles = {}
for target_name in target_names:
    print(target_name)
    tiles[target_name] = roi_tile_from_target_name(target_name)

In [ ]:
sizes = {}
qualities = range(10, 100, 5)

for target_name in target_names:
    sizes[target_name] = []
    for quality in qualities:
        imageio.imsave('test.jpg', tiles[target_name], format='jpg', quality=quality)
        sizes[target_name].append(os.stat('test.jpg').st_size/1024/1024)

In [ ]:
plt.figure(figsize=(8, 6))

for target_name in target_names:
    plt.plot(qualities, sizes[target_name], marker='.', label=target_name)
plt.legend()

plt.gca().set_xlabel('JPG quality')
plt.gca().set_ylabel('Size (MB)')

In [ ]:
# size   condition
# 2.4    no smoothing 90%
# 1.0    no smoothing 70%
# 0.8    smoothing with (1, 1, 1) and 90%
# 1.6    smoothing with (.5, .5, .5) and 90%
# 0.6    smoothing with (.5, .5, .5) and 70%

In [ ]:
plt.imshow((cropped_stack[13, :, :]/255)**.3)